In [ ]:
import numpy as np
import math
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sb
import string
from math import pi
from matplotlib import colors
from shapely.geometry import Point, Polygon

%matplotlib inline

pd.options.mode.chained_assignment = None

# Best places to live and work in the U.S. : Climate
This notebook represents the "Climate" section of a larger project.

I'm in my second year of undergrad, so naturally I've been thinking a lot about where to live once I graduate. In this project, I will explore data to augment my decision making. This project was heavily inspired by the list of best places to live by [U.S. Real Estate News](https://realestate.usnews.com/places/rankings/best-places-to-live). Their list is very compelling, and I was curious about their methods. I also found it difficult to compare two cities. So I decided to jump in myself!

Before we get started, I want to mention [weatherspark](https://weatherspark.com/), which is a website that I found while working on this project. They probably have the best climate visualizations out there, so go check it out! 

## Problem definition
What is the profile of each major city in the U.S.? I'll look at climate, population, and economy, as well as other factors that might be subjective and hard to measure (such as recreation, culture, religiosity, sports-ness, and political distribution). The focus of this experiment, compared to the U.S Real Estate News list is:
* Make it easier to compare two or more cities against any parameter or a set of parameters
* Consider how things are changing (e.g. 'What will the cost of living be in 10 years?' instead of 'What is the cost of living now?')
* Investigate other factors that are not covered by U.S. Real Estate News

## Data & cleaning/processing

### Cities
As a starting point, we need a list of cities. The list of metropolitan areas (hitherto referred to as cities) used in this experiment was obtained from [Wikipedia](https://en.wikipedia.org/wiki/List_of_metropolitan_statistical_areas) and downloaded using [wikitable2csv](https://wikitable2csv.ggor.de/). We'll be referring to this list for the rest of the project.

In [ ]:
cities = pd.read_csv(
    '../data/raw/metros.csv',
    usecols=[1, 2], # for climate analysis, let's just pull out the city name and the population
    header=0,
    names=['Metro', 'Pop']
)
cities.head()

I (frustratingly) figured out that some of the metro descriptions actually have en dashes instead of hyphens. This caused some problems, so let's replace those.

In [ ]:
def fix_dashes(city):
    return city['Metro'].replace(u'\u2013', '-') 

cities['Metro'] = cities.apply(
    lambda city: fix_dashes(city), # apply the function defined above
    axis=1 # apply function row-wise
)

### Climate
Climate data are from the [NOAA 1981-2010 normals](https://www.ncdc.noaa.gov/data-access/land-based-station-data/land-based-datasets/climate-normals/1981-2010-normals-data). Several tables from the NOAA normals are used: temperature average, temperature min, temperature max, cloud cover, average windspeed, precipitation, and snowfall. An important thing to note is that the tables do not have column names. The data are labelled by weather station ID, which are inventoried in various files (there are different stations available for different varieties of metrics, e.g. not all stations collect hourly data). This means we're going to have to treat the tables as a relational database, using the station inventory as a lookup table. 

The values in the climate tables are 30-year normals (averages). Each value is actually accurate to one tenth of whatever the unit is (percent, degrees farenheit, inches, MPH, etc), but it is represented as an int. There's also a character that describes the nature of the calculation of the value, but we can disregard that for our purposes. So, 85.4 degrees farenheit will look like `854C` in the tables. This means we're going to have to do some cleaning before we can analyze the data.

In order to connect cities with weather stations, we'll need their latitude and longitude. I retrieved this information from [OpenDataSoft](https://public.opendatasoft.com/explore/dataset/1000-largest-us-cities-by-population-with-geographic-coordinates/table/?sort=-rank&dataChart=eyJxdWVyaWVzIjpbeyJjb25maWciOnsiZGF0YXNldCI6IjEwMDAtbGFyZ2VzdC11cy1jaXRpZXMtYnktcG9wdWxhdGlvbi13aXRoLWdlb2dyYXBoaWMtY29vcmRpbmF0ZXMiLCJvcHRpb25zIjp7InNvcnQiOiItcmFuayJ9fSwiY2hhcnRzIjpbeyJhbGlnbk1vbnRoIjp0cnVlLCJ0eXBlIjoiY29sdW1uIiwiZnVuYyI6IkFWRyIsInlBeGlzIjoicmFuayIsInNjaWVudGlmaWNEaXNwbGF5Ijp0cnVlLCJjb2xvciI6IiNGRjUxNUEifV0sInhBeGlzIjoiY2l0eSIsIm1heHBvaW50cyI6NTAsInNvcnQiOiIifV0sInRpbWVzY2FsZSI6IiIsImRpc3BsYXlMZWdlbmQiOnRydWUsImFsaWduTW9udGgiOnRydWV9). This dataset actually has all the same fields as the Wikipedia data as well, but this dataset is about a decade older so I want to keep the population and population growth data from Wikipedia.

In [ ]:
cities_latlong = pd.read_csv(
    '../data/raw/cities-lat-long.csv',
    sep=';',
    usecols=['City', 'State', 'Coordinates']
)
cities_latlong.head()

Now we can join `cities` and `cities_latlong`. There is one intermediate step - we have to pull out the major city name from `cities.'Metropolitan statistical area'` to join it on `cities_latlong.'City'`. While we're at it, let's pull out the state and put it in its own column.

In [ ]:
def major_city(city):
    """ Given a metro, return the predominant city name. """ 
    metro = city['Metro']
    metro = metro[0: metro.find(',')]
    term = metro.find('-')
    if term == -1:
        return metro
    return metro[0:term]

def state(city):
    """ Given a metro, return the state abbreviation. """
    metro = city['Metro']
    states = metro[metro.find(',') + 2:]
    state = states[0:2]
    if state == 'DC':
        state = 'MD'
    return state

cities['City'] = cities.apply(lambda city: major_city(city), axis=1)
cities['State'] = cities.apply(lambda city: state(city), axis=1)
cities.head()

At this point, we could try to join `cities_latlong` on `'City'`, but we would run into an issue. There are some cities that share a name. For example, Portland OR and Portland ME.

In [ ]:
cities.duplicated(subset='City', keep=False).value_counts()

This means that there are 49 entries using the same name as another city. This would create unexpected results if we joined `cities` with `cities_latlong` on `'City'`. It's tempting to just drop the duplicate rows to avoid the issue, but that leaves out some key cities like Portland ME. Here's another idea: let's change the value for `'City'` of the duplicate with the lower population to the second city in the `'Metropolitan Statistical Area'` column. That won't work either, because many of the problematic cities aren't actually a metro at all, e.g. Springfield MO. It seems like the only option is to somehow join based on the city name AND the state. Fortunately, `cities_latlong` has a column for state. Unfortunately, it's not the two-letter abbreviation that shows up in `cities`. We're going to have to find a way to map the abbreviations to the state names.

I pulled a table from [this website](https://statetable.com/) that will help.

In [ ]:
state_table = pd.read_csv(
    '../data/raw/state-table.csv',
    usecols=[1, 2],
    header=0,
    names=['State name', 'State abbreviation']
)
state_table.head()

In [ ]:
cities = cities.merge(
    state_table,
    how='left',
    left_on='State',
    right_on='State abbreviation'
)
cities.head()

In [ ]:
cities['Duplicate'] = cities.duplicated(subset='City', keep='first') 
# the values are sorted by population, so keep='first' allows us to ignore the more populous city
cities.loc[cities['Duplicate']]

In [ ]:
cities = cities.merge(cities_latlong, how='inner', left_on=['City', 'State name'], right_on=['City', 'State'])
cities.head()

Now our dataframe has lat/long coordinates as well. We should separate the latitude and longitude into 2 columns.

In [ ]:
cities[['Latitude', 'Longitude']] = cities['Coordinates'].str.split(', ', 1, expand=True).astype(float)
cities.dtypes

To keep things tidy, let's remove all the columns that we don't need. I'm also going to remove the 'MSA' from the metro descriptions.

In [ ]:
cities = cities.drop(['City', 'Coordinates', 'State_x', 'State_y', 'Duplicate'], axis=1)
cities['Metro'] = cities.apply(lambda x: x['Metro'][:x['Metro'].find(' MSA')], axis=1)
cities.head()

I'm going to save this version of the table for other analyses.

In [ ]:
cities.to_csv('../data/processed/cities.csv', index=False)

For the rest of the climate investigation, we only need the metro, state abbreviation, lat, and lon.

In [ ]:
cities = cities.drop(['Pop', 'State name'], axis=1)
cities.head()

### Preparing data for analysis
The last step before we can start our investigation is transforming the tables into something usable. The climate tables vary between monthly, daily, and hourly values and are labelled by weather station. I'd like to consolidate all the data into one format and label the data by cities instead of stations. There's also a fair bit of cleaning to take care of. So there are three things to do:
1. **Transform data to a common format.** I think monthly numbers are a sufficient level of fidelity for my purposes, so I can just aggregate all hourly and daily values into monthly values. 
2. **Change labels from stations to cities.** This is a little bit trickier. We're going to have to match cities with stations by minimizing the euclidean distance, calculated from latitude and longitude. 
3. **Clean the data.** During my exploration of the data, I noticed that the NOAA uses negative numbers as NULL placeholders. Different numbers mean different things, but for our purposes, we can just turn them all into actual NULLs.

The code for all this is pretty involved, and there are several tables to process, so I'm going to define a few functions to handle all this.

In [ ]:
def find_station(city, table, stations_file):
    """ 
    Given a row for a city and the stations table filepath,
    return the ID of the closest weather station. 
    """
    
    metro = city['Metro']
    lat, lon = city['Latitude'], city['Longitude']
    state = city['State abbreviation']
    if state == 'DC':
        state = 'MD'
    # FIXME: D.C. creates problems joining the metro table with the weather station table
    #        maybe just hand-pick a weather station for D.C.?
    stations = pd.read_csv(
            stations_file,
            sep='\s+',
            header=None,
            names=['ID', 'LAT', 'LON', '_', 'STATE', 'NAME', '_', '_', '_', '_', '_', '_'],
            usecols=['ID', 'LAT', 'LON', 'STATE']
    )
    stations = stations.merge(
        table,
        how='inner',
        left_on='ID',
        right_on='STATION'
    )
    state_stations = stations[stations['STATE'] == state]
    state_stations['distances'] = state_stations.apply(
        lambda row: (row['LAT'] - lat)**2 + (row['LON'] - lon)**2,
        axis=1
    )
    return state_stations.loc[state_stations['distances'].idxmin()]['ID']

In [ ]:
def mly_to_analysis(table, stations_file):
    """
    Given a monthly climate table, return the table transformed
    to the analysis format.
    """
    # start with a copy of the cities table
    result = cities.copy()
    
    # add a column for the matching weather station
    result['Station'] = result.apply(lambda x: find_station(x, table, stations_file), axis=1)
    
    # the station found may not be in the table, because stations with too many NULLs 
    # will be dropped
        
    # join with climate data
    result = result.merge(
        table,
        how='inner',
        left_on='Station',
        right_on='STATION',
        copy=False
    ).drop(['STATION'], axis=1)
    
    result['Average'] = result.apply(lambda x: x[5:].mean(), axis=1)
    
    return result

def dly_to_analysis(table, stations_file):
    """
    Given a daily climate table, return the table transformed
    to the analysis format.
    TODO
    """
    # start with a copy of the cities table
    result = cities.copy()
    
    # add a column for the matching weather station
    result['Station'] = result.apply(lambda x: find_station(x, table, stations_file), axis=1)
    
    # prepare aggregate column of climate data
    table['Month average'] = table.apply(lambda x: x[2:].mean(), axis=1)
    
    # get the subset of the table that we're interested in
    table = table[['STATION', 'MONTH', 'Month average']]
    
    # pivot so that each month is a column instead of a row, using the average for the value
    table = table.pivot(
        index='STATION',
        columns='MONTH',
        values='Month average'
    )
    
    # join with climate data
    result = result.merge(
        table,
        how='inner',
        left_on='Station',
        right_on='STATION',
        copy=False
    )
    
    result['Average'] = result.apply(lambda x: x[5:].mean(), axis=1)
    
    return result

def hly_to_analysis(table, stations_file):
    """
    Given an hourly climate table, return the table transformed
    to the analysis format.
    TODO
    """
    
    # start with a copy of the cities table
    result = cities.copy()

    # add a column for the matching weather station
    result['Station'] = result.apply(lambda x: find_station(x, table, stations_file), axis=1)

    # prepare aggregate column of climate data
    table['Day average'] = table.apply(lambda x: x[3:].mean(), axis=1)

    # get the subset of the table that we're interested in
    table = table[['STATION', 'MONTH', 'DAY', 'Day average']]

    # aggregate day averages 
    table = table.groupby(by=['STATION', 'MONTH']).agg('mean').drop(['DAY'], axis=1).reset_index().pivot(
        index='STATION',
        columns='MONTH',
        values='Day average'
    )

    # join with climate data
    result = result.merge(
        table,
        how='inner',
        left_on='Station',
        right_on='STATION',
        copy=False
    )    
    
    result['Average'] = result.apply(lambda x: x[5:].mean(), axis=1)
    
    return result    

Now we just need a function to read an NOAA file and turn it into a dataframe, so we can pass that dataframe to the function above.

In [ ]:
# map of integers to month descriptions
months_map = {
    1: 'January',
    2: 'February',
    3: 'March',
    4: 'April',
    5: 'May',
    6: 'June',
    7: 'July',
    8: 'August',
    9: 'September',
    10: 'October',
    11: 'November',
    12: 'December'
}

def _clean(value, mag):
    """
    Define the behavior to clean values;
    to be used in read_climate_table.
    """
    # remove letters
    value = int(str(value).strip(string.ascii_letters)) / mag
    
    # treat missing values as NULLs
    if (value == -9999 / mag or
        value == -8888 / mag or
        value == -7777 / mag or 
        value == -6666 / mag or
        value == -5555 / mag):
        value = None
    return value

def read_climate_table(file, inv, form='None'):
    """ 
    Return a dataframe from a NOAA
    given a filepath and a format, 
    either 'mly', 'dly', or 'hly'.
    """
    
    cols = [
        'STATION',
        1, 2, 3, 4,
        5, 6, 7, 8,
        9, 10, 11, 12
    ] # numbers are months
    
    # describes the column at which the climate measurements start
    # preceding this column is station (and month (and day))
    values_start = 1
    
    if form == 'dly':
        cols = [
            'STATION', 'MONTH',
            1,2,3,4,5,6,7,8,9,10,
            11,12,13,14,15,16,17,18,19,20,
            21,22,23,24,25,26,27,28,29,30,
            31
        ] # numbers are days of the month
        values_start = 2
        
    if form == 'hly':
        cols = [
            'STATION', 'MONTH', 'DAY',
            1,2,3,4,5,6,7,8,
            9,10,11,12,13,14,15,16,
            17,18,19,20,21,22,23,24
        ] # numbers are hours of the day
        values_start = 3
    
    # read the table file
    df = pd.read_table(
        file,
        sep='\s+',
        header=None,
        names=cols
    )
    
    # values are represented in tenths (or 100ths for the precipitation file)
    if 'prcp' in file and 'ge' not in file:
        mag = 100
    else:
        mag = 10
        
    # apply _clean to all values except weather station
    df.iloc[:, values_start:] = df.iloc[:, values_start:].apply(
        lambda x: x.apply(
            lambda y: _clean(y, mag)
        ), axis=1
    )
    
    # drop stations with too many missing values
    if form == 'dly':
        df = df.dropna(thresh=4)
    else:
        df = df.dropna()
         
    # convert to analysis format
    if form == 'mly':
        df =  mly_to_analysis(df, inv)
    elif form == 'dly':
        df = dly_to_analysis(df, inv)
    elif form == 'hly':
        df = hly_to_analysis(df, inv)
        
    # lastly, rename months columns
    return df.rename(mapper=months_map, axis=1)

Now, we can read the NOAA data into dataframes.

In [ ]:
# daily high temperature
temp_hi = read_climate_table(
    '../data/raw/climate/dly-tmax-normal.txt',
    '../data/raw/climate/temp-inventory.txt',
    form='dly'
)

# daily low temperature
temp_lo = read_climate_table(
    '../data/raw/climate/dly-tmin-normal.txt',
    '../data/raw/climate/temp-inventory.txt',
    form='dly'
)

# monthly rainfall in inches
rain = read_climate_table(
    '../data/raw/climate/mly-prcp-normal.txt',
    '../data/raw/climate/prcp-inventory.txt',
    form='mly'
)

# daily probability of 1 or more inches of rain
# rain_prob = read_climate_table(
#     '../data/raw/climate/dly-prcp-pctall-ge100hi.txt',
#     '../data/raw/climate/prcp-inventory.txt',
#     form='dly'
# )

# monthly snowfall in inches
snow = read_climate_table(
    '../data/raw/climate/mly-snow-normal.txt',
    '../data/raw/climate/prcp-inventory.txt',
    form='mly'
)

# daily probability of 1 or more inches of snow
# snow_prob = read_climate_table(
#     '../data/raw/climate/dly-snow-pctall-ge010ti.txt',
#     '../data/raw/climate/hly-inventory.txt',
#     form='dly'
# )


# hourly dewpoint temperature
dewpoint = read_climate_table(
    '../data/raw/climate/hly-dewp-normal.txt',
    '../data/raw/climate/hly-inventory.txt',
    form='hly'
)


# hourly overcast percentage
clouds = read_climate_table(
    '../data/raw/climate/hly-clod-pctovc.txt',
    '../data/raw/climate/hly-inventory.txt',
    form='hly'
)


# hourly windspeed
wind = read_climate_table(
    '../data/raw/climate/hly-wind-avgspd.txt',
    '../data/raw/climate/hly-inventory.txt',
    form='hly'
)

Now, I'm going to save each of these so I never have to worry about that ever again.

In [ ]:
temp_hi.to_csv('../data/processed/temp-hi.csv', index=False)
temp_lo.to_csv('../data/processed/temp-lo.csv', index=False)
rain.to_csv('../data/processed/rain.csv', index=False)
snow.to_csv('../data/processed/snow.csv', index=False)
dewpoint.to_csv('../data/processed/dewpoint.csv', index=False)
clouds.to_csv('../data/processed/clouds.csv', index=False)
wind.to_csv('../data/processed/wind.csv', index=False)

Now, in future runs, I can simply load these tables from the files I just created.

In [ ]:
# temp_hi = pd.read_csv('../data/processed/temp-hi.csv')
# temp_lo = pd.read_csv('../data/processed/temp-lo.csv')
# rain = pd.read_csv('../data/processed/rain.csv')
# snow = pd.read_csv('../data/processed/snow.csv')
# dewpoint = pd.read_csv('../data/processed/dewpoint.csv')
# clouds = pd.read_csv('../data/processed/clouds.csv')
# wind = pd.read_csv('../data/processed/wind.csv')

## Investigation
Now we're ready to do some investigation.

### Mapping features
Let's make some maps to help our brains process the climate data. We'll do this with GeoPandas.

We'll need a shapefile map of the US. I'm using the states_21basic map from [ARC GIS](https://www.arcgis.com/home/item.html?id=f7f805eb65eb4ab787a0a3e1116ca7e5). I learned about how to use the map from [this article](https://medium.com/@erikgreenj/mapping-us-states-with-geopandas-made-simple-d7b6e66fa20d).

In [ ]:
import geopandas as gpd

usa = gpd.read_file('../data/raw/maps/states_21basic/states.shp')

# let's drop hawaii and alaska, because they make plotting very difficult
contig_usa = usa.drop(
    usa[usa['STATE_NAME'] == 'Hawaii'].index
).drop(
    usa[usa['STATE_NAME'] == 'Alaska'].index
)

#contig_usa will be passed to the plot function
contig_usa.head()

In order to plot the cities with geopandas, we'll have to turn the DataFrame into a GeoDataFrame. You can learn how to do this from the [GeoPandas documentation](https://geopandas.readthedocs.io/en/latest/gallery/create_geopandas_from_pandas.html). Learn about colormaps from the [matplotlib documentation](https://matplotlib.org/users/colormaps.html#colorcet).

In [ ]:
def climate_gdf(df=None):
    """
    Return a GDF for geospatial plotting from a regular dataframe with latitude and longitude columns.
    """
    df['Coordinates'] = list(zip(df['Longitude'], df['Latitude']))
    df['Coordinates'] = df['Coordinates'].apply(Point)

    # make the GDF
    # exclude Alaska because it is hard to put on a map
    gdf = gpd.GeoDataFrame(df[~(df['State abbreviation'] == 'AK')], geometry='Coordinates')
    return gdf

I'm going to define a function that handles the mapping. I wanted to separate the data by month, so I'm using a widget to allow for month selection. 

In [ ]:
import ipywidgets as widgets
from ipywidgets import interact, interactive, fixed, interact_manual

def _climate_plot(gdf=None, month=1, cmap='Greens', base=None):
    w = widgets.IntSlider(
        value=1,
        min=1,
        max=12,
        step=1,
        description='Month:',
        disabled=False,
        continuous_update=False,
        orientation='horizontal',
        readout=True,
        readout_format='d'
    )
    
    fig, ax = plt.subplots(1, 1, figsize=(10,6), dpi=200)
    sb.set(style='white')
    ax.tick_params(
        axis='both',
        which='both',
        bottom=False,
        top=False,
    )
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
    base_plot = base.plot(color='0.6', edgecolor='white', ax=ax)
    layer = gdf.plot(
        markersize=20,
        ax=ax,
        alpha=1,
        column=months_map[month],
        cmap=cmap,
        legend=True,
        vmin=gdf.iloc[:,5:17].min().min(), vmax=gdf.iloc[:,5:17].max().max()
    )
    plt.title(months_map[month])
    ax.axis('off')
    plt.show()
    
def climate_plot(gdf=None, cmap='Greens', base=None):
    interact_manual(
        _climate_plot,
        month=widgets.IntSlider(min=1,max=12,step=1,value=1),
        gdf=fixed(gdf),
        cmap=fixed(cmap),
        base=fixed(base)
    )

In [ ]:
climate_plot(
    gdf=climate_gdf(df=clouds),
    base=contig_usa,
    cmap='viridis_r'
)

## City profiling
Let's try taking a look at individual cities. I currently live in Fort Collins, Colorado.

In [ ]:
# TODO

## Modeling
In order to use the climate evaluation as a parameter in this experiment, I think it would be best to come up with a climate classifier. Then we can add a new column to our data, `'Region'`, that will give us a good idea of what the weather is like there. 

The climate data are not labelled (i.e. there is no existing field `'Region'` describing cities that we could use to train a model), so we'll have to use an unsupervised learning technique. Let's try K means clustering from SciKit-Learn.

In [ ]:
from sklearn.cluster import KMeans
from sklearn.mixture import GaussianMixture
from sklearn import preprocessing

In [ ]:
def climate_normalize(df):
    sub = df.iloc[:,5:18]
    array = sub.values
    scaler = preprocessing.MinMaxScaler()
    array_scaled = scaler.fit_transform(array)
    result = pd.DataFrame(
        array_scaled,
        columns=df.iloc[:,5:18].columns
    )
    result['Metro'] = df['Metro']
    return result

temp_hi_norm = climate_normalize(temp_hi)
temp_lo_norm = climate_normalize(temp_lo)
rain_norm = climate_normalize(rain)
snow_norm = climate_normalize(snow)
dewpoint_norm = climate_normalize(dewpoint)
clouds_norm = climate_normalize(clouds)
wind_norm = climate_normalize(wind)

In [ ]:
# make a new df for clustering,
# dropping all the columns except 
# for the features that we are going 
# to use for the model
cities_m = cities.copy()

cities_m = cities_m.merge(
    temp_hi[['Metro', 'Average']],
    how='inner', on='Metro'
).rename(mapper={'Average': 'Temp high average'}, axis=1)
cities_m = cities_m.merge(
    temp_lo[['Metro', 'Average']],
    how='inner', on='Metro'
).rename({'Average': 'Temp low average'}, axis=1)
cities_m = cities_m.merge(
    rain[['Metro', 'Average']],
    how='inner', on='Metro'
).rename({'Average': 'Rainfall average'}, axis=1)
cities_m = cities_m.merge(
    snow[['Metro', 'Average']],
    how='inner', on='Metro'
).rename({'Average': 'Snowfall average'}, axis=1)
cities_m = cities_m.merge(
    dewpoint[['Metro', 'Average']],
    how='inner', on='Metro'
).rename({'Average': 'Dewpoint average'}, axis=1)
cities_m = cities_m.merge(
    clouds[['Metro', 'Average']],
    how='inner', on='Metro'
).rename({'Average': 'Overcast average'}, axis=1)
cities_m = cities_m.merge(
    wind[['Metro', 'Average']],
    how='inner', on='Metro'
).rename({'Average': 'Windspeed average'}, axis=1)

cities_m.head()

At this point I wonder, should latitude and longitude be used in the model? This would encourage the climate classification to classify regions, not just similar weather patterns.

Before we can do any clustering, we have to normalize the data.

In [ ]:
sub = cities_m.iloc[:,4:]
metros = cities_m['Metro']
array = sub.values
scaler = preprocessing.MinMaxScaler()
array_scaled = scaler.fit_transform(array)
cities_m = pd.DataFrame(
    array_scaled,
    columns=sub.columns
)
cities_m['Metro'] = metros
cities_m.head()

In [ ]:
sb.set(style='white')
sb.pairplot(data=cities_m)
fig = plt.gcf()
plt.show()

In [ ]:
# fig.savefig('../output/figures/pairplot.png', dpi=800, bbox_inches='tight')

In [ ]:
N = 8 # we have to specify the number of clusters
model = KMeans(n_clusters=N) 
model.fit(cities_m.iloc[:,0:-1].values) # the model takes a NumPy array as input
climate_clusters = model.predict(cities_m.iloc[:,0:-1].values)

The output of predict() is a NumPy 1D array containing the class for each row of the dataframe. This means we can just add this array to the dataframe as a new column.

In [ ]:
cities_m['Climate cluster'] = climate_clusters # add to the normalized array for evaluation later
cities['Climate cluster'] = climate_clusters
cities.head()

I got the code for this discrete color map function from [github.com/jakevdp](https://gist.github.com/jakevdp/91077b0cae40f8f8244a).

In [ ]:
def discrete_cmap(N, base_cmap=None):
    """Create an N-bin discrete colormap from the specified input map"""

    base = plt.cm.get_cmap(base_cmap)
    color_list = base(np.linspace(0, 1, N))
    cmap_name = base.name + str(N)
    if base_cmap == None:
        return plt.cm.get_cmap(base_cmap, N)
    return base.from_list(cmap_name, color_list, N)

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(10,6), dpi=200)
sb.set(style='white')
ax.tick_params(
    axis='both',
    which='both',
    bottom=False,
    top=False,
)
ax.get_xaxis().set_visible(False)
ax.get_yaxis().set_visible(False)
base_plot = contig_usa.plot(color='0.7', edgecolor='w', ax=ax)
layer = climate_gdf(cities).plot(
    markersize=20,
    ax=ax,
    alpha=1,
    column='Climate cluster',
    cmap=discrete_cmap(N, base_cmap='gist_rainbow'),
    legend=True
)
ax.axis('off')
plt.show()

In [ ]:
# fig.savefig('../output/figures/clustermap.png', dpi=800, bbox_inches='tight')

Let's make an attempt to describe the climate clusters. To do so, we'll have to look at each cluster's distribution of each climate variable.

In [ ]:
g = sb.FacetGrid(cities_m, col='Climate cluster')
g.map(sb.kdeplot, 'Temp high average')

g = sb.FacetGrid(cities_m, col='Climate cluster')
g.map(sb.kdeplot, 'Temp low average')

g = sb.FacetGrid(cities_m, col='Climate cluster')
g.map(sb.kdeplot, 'Rainfall average')

g = sb.FacetGrid(cities_m, col='Climate cluster', ylim=100)
g.map(sb.kdeplot, 'Snowfall average')

g = sb.FacetGrid(cities_m, col='Climate cluster')
g.map(sb.kdeplot, 'Dewpoint average')

g = sb.FacetGrid(cities_m, col='Climate cluster')
g.map(sb.kdeplot, 'Overcast average')

g = sb.FacetGrid(cities_m, col='Climate cluster')
g.map(sb.kdeplot, 'Windspeed average')

plt.show()

These plots describe the climate clusters. Let's create climate descriptors based on the distributions of climate variables.

In [ ]:
cluster_map = {
    0: 'warm/windy',
    1: 'cool/cloudy/windy',
    2: 'temperate/rainy/calm',
    3: 'cool/dry',
    4: 'hot/rainy/sticky',
    5: 'cold/snowy/cloudy',
    6: 'warm/sunny/calm',
    7: 'cool/rainy/cloudy',
}
cities['Climate'] = cities.apply(lambda x: cluster_map[x['Climate cluster']], axis=1)
cities = cities.drop(['Climate cluster'], axis=1)
cities.head()

## Ranking cities
The whole point of this experiment is to figure out which cities have the best climate. We can do this by coming up with some ideal climate as a point in n-dimensional space, where n is the number of variables we've described. Then, we can rank cities in ascending order by their distance from this point. The cities closest to the point have the best climate.

When we calculate the distance, we're going to want to use the normalized values. Otherwise, a difference along the `cloud` axis might be more significant than a difference along the `wind` axis, because the `cloud` values vary much more.

The question is, what is the ideal climate? This is extremely subjective and even though I have a pretty good idea of weather that I like and don't like, I won't be able to come up with exact numbers to describe this. With all this in mind, I'll try my best.

In [ ]:
ideal = {
    'temp_hi': 0.5,
    'temp_lo': 0.4,
    'rain': 0.7,
    'dewpoint': 0.5,
    'snow': 0.2,
    'clouds': 0.5,
    'wind': 0
}

def evaluate_climate(city, ideal):
    """
    Given a city name and a dict describing the ideal climate,
    return the distance of that city in n-dimensional 
    space from the point representing the ideal climate.
    """
    edges = []
    edges.append((ideal['temp_hi']  - temp_hi_norm[temp_hi_norm['Metro'] == city].iloc[0]['Average']) ** 2)
    edges.append((ideal['temp_lo']  - temp_lo_norm[temp_lo_norm['Metro'] == city].iloc[0]['Average']) ** 2)
    edges.append((ideal['rain']     - rain_norm[rain_norm['Metro'] == city].iloc[0]['Average']) ** 2)
    edges.append((ideal['snow']     - snow_norm[snow_norm['Metro'] == city].iloc[0]['Average']) ** 2)
    edges.append((ideal['dewpoint'] - dewpoint_norm[dewpoint_norm['Metro'] == city].iloc[0]['Average']) ** 2)
    edges.append((ideal['clouds']   - clouds_norm[clouds_norm['Metro'] == city].iloc[0]['Average']) ** 2)
    edges.append((ideal['wind']     - wind_norm[wind_norm['Metro'] == city].iloc[0]['Average']) ** 2)
   
    return sum(edges)

In [ ]:
cities['Score'] = cities.apply(lambda x: evaluate_climate(x['Metro'], ideal), axis=1)
cities.head(50).sort_values('Score', ascending=True).head(10)['Metro']

I'm going to save a copy of this dataframe with the climate rankings.

In [ ]:
cities.sort_values(
    'Score', ascending=True
).reset_index().rename(
    mapper={'index': 'Climate rank'}, axis=1
).drop(
    ['Score', 'Coordinates'], axis=1
).to_csv('../data/processed/climate-rankings.csv')

Finally, I'm going to map the climate scores.

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(10,6), dpi=200)
sb.set(style='white')
ax.tick_params(
    axis='both',
    which='both',
    bottom=False,
    top=False,
)
ax.get_xaxis().set_visible(False)
ax.get_yaxis().set_visible(False)
base_plot = contig_usa.plot(color='0.7', edgecolor='white', ax=ax)
layer = climate_gdf(cities).plot(markersize=20, ax=ax, alpha=1, column='Score', cmap='RdYlGn_r', legend=False)
ax.axis('off')
plt.show()

In [ ]:
# fig.savefig('../output/figures/climate-rank-map.png', dpi=800, bbox_inches='tight')